In [4]:
# Import required libraries
import os
import cv2
import shutil
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [5]:
# Define paths
original_dataset_dir = 'PetImages'
base_dir = 'PetImages_split'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Dog'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'Dog'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'Cat'), exist_ok=True)

# Function to remove corrupted images
def remove_corrupted_images(directory):
    for category in ['Dog', 'Cat']:
        folder_path = os.path.join(directory, category)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                img = Image.open(file_path)
                img.verify()
            except (IOError, SyntaxError):
                os.remove(file_path)
                print(f"Removed corrupted image: {file_path}")

# Calling the function  -- created by nithishkumar
remove_corrupted_images(original_dataset_dir)

def split_and_copy_files(category, split_ratio=0.2):
    files = [f for f in os.listdir(os.path.join(original_dataset_dir, category)) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    train_files, val_files = train_test_split(files, test_size=split_ratio, random_state=42)
    
    for file in train_files:
        shutil.copy(os.path.join(original_dataset_dir, category, file), os.path.join(train_dir, category, file))
        
    for file in val_files:
        shutil.copy(os.path.join(original_dataset_dir, category, file), os.path.join(val_dir, category, file))


# Separate images into training and validation sets
split_and_copy_files('Dog')
split_and_copy_files('Cat')


In [8]:
# Data preprocessing
img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=20)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Calculate steps per epoch and validation steps //authour-Nithishkumar
steps_per_epoch = train_generator.samples // batch_size
validation_steps = val_generator.samples // batch_size

# Custom generator to handle corrupted images
def custom_image_generator(generator):
    while True:
        try:
            yield next(generator)
        except Exception as e:
            print(f"Skipping a corrupted image: {e}")

# Wrapping generators to handle corrupted images
train_generator = custom_image_generator(train_generator)
val_generator = custom_image_generator(val_generator)

# Load and prepare the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Found 10141 images belonging to 2 classes.
Found 2941 images belonging to 2 classes.


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 230, 230, 3)       │               0 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 112, 112, 64)      │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 112, 112, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 112, 112, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 114, 114, 64)      │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 56, 56, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 56, 56, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 56, 56, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 24,112,770 (91.98 MB)

 Trainable params: 525,058 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [22]:

# Define the training function
def train_initial_model():
    try:
        # Train the model
        history = model.fit(
            train_generator,
            epochs=5,
            steps_per_epoch=steps_per_epoch,
            validation_data=val_generator,
            validation_steps=validation_steps
        )
        
        # Save the initial model
        model.save('fine_tuned_resnet50_initial.h5')
        
    except tf.errors.InvalidArgumentError as e:
        print(f'TensorFlow InvalidArgumentError: {e}')
    except tf.errors.OpError as e:
        print(f'TensorFlow OpError: {e}')
    except Exception as e:
        print(f'An unexpected error occurred: {e}')

# Start initial training
train_initial_model()
print("traininng completed")

Epoch 1/5
 39/316 ━━━━━━━━━━━━━━━━━━━━ 28:03 6s/step - accuracy: 0.6047 - loss: 0.6561Skipping a corrupted image: cannot identify image file <_io.BytesIO object at 0x0000016F27050360>
316/316 ━━━━━━━━━━━━━━━━━━━━ 2427s 8s/step - accuracy: 0.6124 - loss: 0.6574 - val_accuracy: 0.4823 - val_loss: 0.7232
Epoch 2/5
 91/316 ━━━━━━━━━━━━━━━━━━━━ 11:43 3s/step - accuracy: 0.5897 - loss: 0.6565Skipping a corrupted image: cannot identify image file <_io.BytesIO object at 0x0000016F27083B00>
316/316 ━━━━━━━━━━━━━━━━━━━━ 1267s 4s/step - accuracy: 0.5998 - loss: 0.6565 - val_accuracy: 0.6597 - val_loss: 0.6355
Epoch 3/5
239/316 ━━━━━━━━━━━━━━━━━━━━ 4:01 3s/step - accuracy: 0.6186 - loss: 0.6570Skipping a corrupted image: cannot identify image file <_io.BytesIO object at 0x0000016F27081530>
316/316 ━━━━━━━━━━━━━━━━━━━━ 1251s 4s/step - accuracy: 0.6167 - loss: 0.6565 - val_accuracy: 0.6645 - val_loss: 0.6172
Epoch 4/5
222/316 ━━━━━━━━━━━━━━━━━━━━ 4:51 3s/step - accuracy: 0.6195 - loss: 0.6548Skippin

traininng completed


In [24]:
model.save('my_model.keras')
print("model saved")

model saved
